In [1]:
# Load required libraries

import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
# Read the csv data into a dataframe
loan_df=pd.read_csv("/Users/manikantanvenkataraman/Downloads/loan_train.csv")
loan_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
# Drop the column loan id

loan_df.drop(['Loan_ID'],axis = 1, inplace=True)
loan_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [8]:
# Drop rows with missing data

loan_df.dropna(inplace=True)
loan_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [9]:
# Convert categorical features into dummy variables

dummies = pd.get_dummies(loan_df[['Gender', 'Married', 'Education', 'Self_Employed','Property_Area', 'Loan_Status']], drop_first=True)
loan_df = pd.concat([loan_df.drop(['Gender', 'Married', 'Education', 'Self_Employed','Property_Area', 'Loan_Status'],axis=1), dummies],axis=1)
loan_df.head()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
1,1,4583,1508.0,128.0,360.0,1.0,1,1,0,0,0,0,0
2,0,3000,0.0,66.0,360.0,1.0,1,1,0,1,0,1,1
3,0,2583,2358.0,120.0,360.0,1.0,1,1,1,0,0,1,1
4,0,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,1,1
5,2,5417,4196.0,267.0,360.0,1.0,1,1,0,1,0,1,1


In [55]:
# Split the data into train and test

from sklearn.model_selection import train_test_split

x= loan_df[['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Education_Not Graduate','Self_Employed_Yes', 'Property_Area_Urban']]
y=loan_df[['Loan_Status_Y']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(384, 8) (96, 8) (384, 1) (96, 1)


In [57]:
# Pipeline with a min-max scaler and a KNN classifier 

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.model_selection import GridSearchCV


# Create standardizer

scaler= MinMaxScaler()

#Create a KNN Classifier

knn=KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

#Create a pipleline

pipe= Pipeline([("standardizer", scaler), ("knn", knn)])


pipe.fit(x_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Pipeline(steps=[('standardizer', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

In [58]:
# Accuracy on the test set

pipe.score(x_test,y_test)

0.7291666666666666

In [62]:
# search space 

search_space=[{"knn__n_neighbors": [1,2,3,4,5,6,7,8,9,10]}]

In [63]:
# Create grid search

gs_knn=GridSearchCV(pipe, search_space, cv=5, verbose=0)
gs_knn.fit(x_train, y_train)
gs_knn.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

{'knn__n_neighbors': 5}

In [64]:
# Best Model Score

gs_knn.score(x_test, y_test)

0.7291666666666666

In [65]:
# Knn with logistic regression and Random Forest

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

pipe=Pipeline([("classifier", RandomForestClassifier())])

search_space=[{"classifier": [LogisticRegression()], "classifier__penalty":['11','12'], "classifier__C": np.logspace(0,4,10)}, {"classifier": [RandomForestClassifier()], "classifier__n_estimators":[10,100,1000], "classifier__max_features":[1,2,3]}]

#Grid Search
gridsearch=GridSearchCV(pipe, search_space, cv=5, verbose=0)

#Fit grid search
gridsearch.fit(x_train, y_train)
gridsearch.best_params_


/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

{'classifier': RandomForestClassifier(max_features=3),
 'classifier__max_features': 3,
 'classifier__n_estimators': 100}

In [66]:
# Best Model Score

gridsearch.score(x_test, y_test)

0.7708333333333334

                              Analysis Summary
                              
                              1. The pipleine automates the model workflow by preprocessing of the data and representation of the predictor in a single step. The pipeline created with minmax scaler and knn on the data set initially produced a model accuracy of 72.9166%.
                              
                              2. Gridserachcv is used for hyperparameter tuning. It runs through all the parameters and determine the best parameter. The model accuracy with GridsearchCV remained the same as 72.9166%.
                              
                              3. Gridsearchcv is also used for best model selection. It selects the best model using cross validation. For the given dataset, gridsearchcv on bith logistinc regression and random classifier resulted in the best model score off 77.0833% 